init variables

In [1]:
from diffusers.utils import load_image
from pathlib import Path

import torch

PROMPT = "cozy vibe"
CACHE_DIR="Z:\MLL_HG"
IMAGE = load_image("./pics/rendered3d.png")

out_dir = Path("ai_pics")

# checkpoint for generating Maps
checkpoint = "lllyasviel/Annotators"

generator = torch.manual_seed(0)


pipeline for normal map

In [2]:
from controlnet_aux import NormalBaeDetector

controlnet = NormalBaeDetector.from_pretrained(
  checkpoint,
  cache_dir=CACHE_DIR,
)
    
normalMap = controlnet(IMAGE)


a:\work\conda\envs\py3119\Lib\site-packages\controlnet_aux\mediapipe_face\mediapipe_face_common.py:7: UserWarning: The module 'mediapipe' is not installed. The package will have limited functionality. Please install it using the command: pip install 'mediapipe'
  warnings.warn(
a:\work\conda\envs\py3119\Lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
a:\work\conda\envs\py3119\Lib\site-packages\timm\models\registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
a:\work\conda\envs\py3119\Lib\site-packages\controlnet_aux\segment_anything\modeling\tiny_vit_sam.py:654: UserWarning: Overwriting tiny_vit_5m_224 in registry with contr

Loading base model ()...Done.
Removing last two layers (global_pool & classifier).


a:\work\conda\envs\py3119\Lib\site-packages\controlnet_aux\normalbae\__init__.py:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(fpath, map_location='cpu

pipeline for depth map

In [3]:
import numpy as np
from transformers import pipeline
from PIL import Image

depth_estimator = pipeline('depth-estimation')
dImage = depth_estimator(IMAGE)['depth']
dImage = np.array(dImage)
dImage = dImage[:, :, None].astype(np.uint8)
dImage = np.concatenate([dImage, dImage, dImage], axis=2)
depthMap = Image.fromarray(dImage)



No model was supplied, defaulted to Intel/dpt-large and revision bc15f29 (https://huggingface.co/Intel/dpt-large).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of DPTForDepthEstimation were not initialized from the model checkpoint at Intel/dpt-large and are newly initialized: ['neck.fusion_stage.layers.0.residual_layer1.convolution1.bias', 'neck.fusion_stage.layers.0.residual_layer1.convolution1.weight', 'neck.fusion_stage.layers.0.residual_layer1.convolution2.bias', 'neck.fusion_stage.layers.0.residual_layer1.convolution2.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a 

Ganerating images with maps

In [4]:
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
checkpoint = "lllyasviel/control_v11f1p_sd15_depth"
controlnet = ControlNetModel.from_pretrained(checkpoint, cache_dir=CACHE_DIR, torch_dtype=torch.float16)

pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", controlnet=controlnet, cache_dir=CACHE_DIR, torch_dtype=torch.float16
).to("cuda")

imageDepth = pipe(PROMPT, num_inference_steps=25, generator=generator, image=depthMap).images[0]
pipe.enable_model_cpu_offload() 


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


  0%|          | 0/25 [00:00<?, ?it/s]

In [5]:

checkpoint = "lllyasviel/control_v11p_sd15_normalbae"
controlnet = ControlNetModel.from_pretrained(checkpoint, cache_dir=CACHE_DIR, torch_dtype=torch.float16)

pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", controlnet=controlnet, cache_dir=CACHE_DIR, torch_dtype=torch.float16
).to("cuda")

imageNormal = pipe(PROMPT, num_inference_steps=25, generator=generator, image=normalMap).images[0]
pipe.enable_model_cpu_offload()



Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

Saving images

In [8]:
from diffusers.utils import make_image_grid
target = (512, 265)
images = [IMAGE,IMAGE, depthMap, normalMap, imageDepth, imageNormal ]
images = [img.resize(target, Image.LANCZOS) for img in images]
make_image_grid(images, rows=3, cols=2).save(out_dir / "AI_exports.jpg")